<a href="https://colab.research.google.com/github/ParalelaUCM/biciMAD/blob/master/AsesorDeRutaBiciMAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

Instalamos y configuramos las herramientas necesarias para empezar a trabajar

In [0]:
!apt-get install openjdk-8-jdk
"""
#!apt install unzip ya no lo usamos
"""
!apt-get install unrar
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install pyspark

In [0]:
import json
import os
import matplotlib.pyplot as plt
import statistics as stats
from pyspark import SparkContext

In [0]:
sc = SparkContext()

# **Descargamos los datasets de bicimad**
Descargaremos cada dataset de la web oficial de biciMAD.
La función descarga usa los paquete Beautiful Soup y request, automatizando la obtención de los archivos. Los renombraremos a '.zip' o '.rar' para poder descomprimirlos y una vez descomprimidos los movemos a la carpeta 'datasets'. Por último eliminamos los ficheros comprimidos.

In [0]:
#Estructura de carpetas para el dataset
!mkdir dataset
!cd dataset/
!mkdir dataset/usages
!mkdir dataset/stations

In [0]:
!pip install patool

from bs4 import BeautifulSoup
import requests
import os
import zipfile
import patoolib

#datos de páginas web para la descarga de los dataset
pagina = "https://opendata.emtmadrid.es"
url = requests.get("https://opendata.emtmadrid.es/Datos-estaticos/Datos-generales-(1)")
html_doc = url.text
soup = BeautifulSoup(html_doc, 'html.parser')
meses_l = [None, "enero","febrero","marzo","abril","mayo","junio","julio",
           "agosto","septiembre","octubre","noviembre","diciembre"]

In [0]:
def descarga(mes,año):
  for link in soup.find_all('a'):
    l = link.get('title')
    if isinstance(l,str):
      if meses_l[mes] in l.lower() and str(año) in l:
        if "uso" in l:
          print(l)
          enlace = link.get('href')
          url = pagina+enlace
          r = requests.get(url, allow_redirects=True)
          open('temp.aspx', 'wb').write(r.content)
          os.rename("temp.aspx","temp.zip")
          try:
            with zipfile.ZipFile("/content/temp.zip", 'r') as zip_ref:
              zip_ref.extractall("/content/dataset/usages")
            os.remove("/content/temp.zip")
          except:
            os.rename("temp.zip","temp.rar")
            patoolib.extract_archive("temp.rar", outdir="/content/dataset/usages")
            os.remove("/content/temp.rar")
        elif "estaciones" in l:
          print(l)
          enlace = link.get('href')
          url = pagina+enlace
          r = requests.get(url, allow_redirects=True)
          open('temp.aspx', 'wb').write(r.content)
          os.rename("temp.aspx","temp.zip")
          try:
            with zipfile.ZipFile("/content/temp.zip", 'r') as zip_ref:
              zip_ref.extractall("/content/dataset/stations")
            os.remove("/content/temp.zip")
          except:
            os.rename("temp.zip","temp.rar")
            patoolib.extract_archive("temp.rar", outdir="/content/dataset/stations")
            os.remove("/content/temp.rar")

In [7]:
for i in range(1,7):
  descarga(i,2019)

Datos de uso de Enero de 2019. Nueva ventana
Situación estaciones bicimad por día y hora de Enero de 2019. Nueva ventana
Datos de uso de Febrero de 2019. Nueva ventana
Situación estaciones bicimad por día y hora de Febrero de 2019. Nueva ventana
patool: Extracting temp.rar ...
patool: running /usr/bin/unrar x -- /content/temp.rar
patool:     with cwd='/content/dataset/stations'
patool: ... temp.rar extracted to `/content/dataset/stations'.
Datos de uso de Marzo de 2019. Nueva ventana
Situación estaciones bicimad por día y hora de Marzo de 2019. Nueva ventana
patool: Extracting temp.rar ...
patool: running /usr/bin/unrar x -- /content/temp.rar
patool:     with cwd='/content/dataset/stations'
patool: ... temp.rar extracted to `/content/dataset/stations'.
Datos de uso de Abril de 2019. Nueva ventana
Situación estaciones bicimad por día y hora de Abril de 2019. Nueva ventana
patool: Extracting temp.rar ...
patool: running /usr/bin/unrar x -- /content/temp.rar
patool:     with cwd='/content

# **Creamos los RDD**
Una vez tenemos las bases de datos descargadas vamos a codificarlas de forma cómoda.

Para tener los datos almacenados de una forma cómoda primero creamos un diccionario cuya clave va a ser un string con el mes y el año del dataset y como valor va a tener el rdd asociado al uso por usuario de ese mes.

La funcion `mapper_usages` nos servirá para crear la rdd más legible. Nos quedaremos con los datos necesarios y cada linea la codificaremos como un diccionario.

In [0]:
rdd_usages = {} 

In [0]:
def mapper_usages(line):
  data = json.loads(line)
  user = data['user_type']
  user_day = data['user_day_code']
  start = data['idunplug_station']
  end = data['idplug_station']
  date = data['unplug_hourTime']['$date'][0:10]
  hora = data['unplug_hourTime']['$date'][11:19]
  time = data['travel_time']
  age = data['ageRange']
  try:
    track = data['track']
  except:
    track = None
  return {"user_type": user,
          "user_day_code": user_day,
          "start": start,
          "end": end,
          "travel_time": time,
          "date": date,
          "hour": hora,
          "age": age,
          "track": track}

In [16]:
directory = 'dataset/usages'
rdd_usages['2019'] = sc.emptyRDD()
for filename in os.listdir(directory):
    if filename.endswith(".json"):
      #Nos quedamos con la fecha del dataset en formato YYYYMM
      name = filename.split("_")[0]
      rdd_usages[name] = sc.textFile(os.path.join(directory, filename)).map(mapper_usages)
      rdd_usages['2019'] = rdd_usages['2019'].union(rdd_usages[name])
      #DEBUG starts
      print(name)
      #DEBUG ends
    else:
        continue

201906
201905
201901
201904
201902
201903


In [0]:
"""
#Juntamos todos los datos de los meses en uno solo dataset, y lo metemos también al diccionario.
rdd_usages['2019_01a06'] = rdd_usages['201901'].union(rdd_usages['201902'].union(rdd_usages['201903'].union(rdd_usages['201904'].union(rdd_usages['201905'].union(rdd_usages['201906'])))))
#rdd_usages['2019_01a06'] = rdd_usages['201901'].concat(rdd_usages['201902'])
"""

In [19]:
rdd_usages['2019'].take(1)

[{'age': 0,
  'date': '2019-06-01',
  'end': 66,
  'hour': '00:00:00',
  'start': 90,
  'track': None,
  'travel_time': 219,
  'user_day_code': 'e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b',
  'user_type': 1}]

Por otro lado un diccionario cuya clave va a ser tambien un string con el mes y el año del dataset y como valor va a tener el rdd asociado a la ocupación de las estaciones en ese mes.

Aquí usaremos la funcion `mapper_stations` para acomodar los datos.

In [0]:
rdd_stations = {} 

In [0]:
def mapper_stations(line):
  data = json.loads(line)
  day = data['_id'][0:10]
  hour = data['_id'][11:27]
  station = data['stations']
  return {"day": day, "hour": hour, "station": station}

In [22]:
directory = 'dataset/stations'
rdd_stations['2019'] = sc.emptyRDD()
for filename in os.listdir(directory):
    if filename.endswith(".json"):
      #Nos quedamos con la fecha del dataset en formato YYYYMM
      name = filename.split("_")[2].split(".")[0]
      rdd_stations[name] = sc.textFile(os.path.join(directory, filename)).map(mapper_stations)
      rdd_stations['2019'] = rdd_stations['2019'].union(rdd_stations[name])
      #DEBUG starts
      print(name)
      #DEBUG ends
    else:
        continue

201905
201906
201903
201902
201904
201901


# **Asesor de ruta BiciMAD**

Aquí, dada una estación de origen, una de destino y una hora del trayecto, calcularemos el tiempo medio necesitado realizar el trayecto, cuántas bicicletas suele haber disponibles en las estaciones, y si suele haber hueco en la estación final para poder colocar la bicicleta.

In [0]:
import statistics as stats

In [48]:
origen = input('Estación de la que se desea partir: ') #57
destino = input('Estación a la que se desea llegar: ') #38
hora_Viaje = int(input('Hora a la que se desea realizar el viaje(HH): ')) #17

Estación de la que se desea partir: 57
Estación a la que se desea llegar: 38
Hora a la que se desea realizar el viaje(HH): 17


In [49]:
#Funcion que se queda con los viajes alrededor de la hora solicitada
def rangoTiempo(hora, rango, hora_target):
    hora = hora.split(":")
    h = int(hora[0])
    m = int(hora[1])
    hora = h + m/60.0
    print(hora)
    if (hora_target - rango < 0):
      return((hora < hora_target + rango) or 24 + hora_target - rango < hora)
    else:
      return(hora_target - rango < hora and hora < hora_target + rango)

rangoTiempo('17:00:00', 1.5, 18)

17.0


True

In [0]:
#Aquí hacemos un primer filtro quedándonos con todos los trayectos que parten de 
#la estación origen y terminan en la estacion destino
rdd_trayecto = rdd_usages['2019'].filter(lambda x: x['start'] == int(origen) and x['end'] == int(destino))

In [51]:
#Debug
print('Total de viajes similares al del cliente sin tener en cuenta el horario: ', rdd_trayecto.count())

Total de viajes similares al del cliente sin tener en cuenta el horario:  690


In [0]:
#Y ahora filtramos ya solo en el rango de horas especificado. 
#En este caso 1.5h alrededor de la hora elegida
rdd_viaje = rdd_trayecto.filter(lambda x: rangoTiempo(x['hour'], 1.5, hora_Viaje))

In [53]:
#Debug
print('Total de viajes similares al del cliente alrededor de la hora especificada: ', rdd_viaje.count())

Total de viajes similares al del cliente alrededor de la hora especificada:  125


In [54]:
#Estudiemos ahora las medias en tiempo de todos los viajes similares a esa hora
listaTiempos = rdd_viaje.map(lambda x: ([x['travel_time']])).reduce(lambda a, b: a + b)
media_tiempo_viaje = stats.mean(listaTiempos)
print('La media de tiempo empleado por otros usuarios es: ',media_tiempo_viaje, 'segundos')

La media de tiempo empleado por otros usuarios es:  524.024 segundos


In [0]:
def estaciones(lista):
  filtro = []
  for estacion in lista['station']:
    if estacion['number'] == origen or estacion['number'] == destino:
      filtro.append(estacion)
  return {"day":lista['day'],
          "hour":lista['hour'],
          "station":filtro}

"""
rdd_stations['201901a06'] = rdd_stations['201901'].union(rdd_stations['201902'].union(rdd_stations['201903'].union(rdd_stations['201904'].union(rdd_stations['201905'].union(rdd_stations['201906'])))))
"""

#rdd_Sit_fil contiene solo los datos de las dos estaciones,
#la de partida y la de llegada
rdd_Sit_fil = rdd_stations['2019'].map(estaciones)

In [0]:
#Debug
rdd_Sit_fil.take(3)

In [57]:
def calcularBasesLibres(lista, n_estacion):
  for station in lista:
    if station['number'] == n_estacion:
      return station['free_bases']

def calcularBicisLibres(lista, n_estacion):
  for station in lista:
    if station['number'] == n_estacion:
      return station['dock_bikes']

bicisLibres = rdd_Sit_fil.filter(lambda x: rangoTiempo(x['hour'], 1.5, hora_Viaje)).map(lambda x: [calcularBicisLibres(x['station'], origen)]).reduce(lambda a, b: a+b)
basesLibres = rdd_Sit_fil.filter(lambda x: rangoTiempo(x['hour'], 1.5, media_tiempo_viaje / 3600.0)).map(lambda x: [calcularBasesLibres(x['station'], destino)]).reduce(lambda a, b: a+b)

totalBasesPorEstacion = {}
stationsLst = rdd_Sit_fil.take(1)[0]['station']
totalBasesPorEstacion[stationsLst[0]['number']] = stationsLst[0]['total_bases']
totalBasesPorEstacion[stationsLst[1]['number']] = stationsLst[1]['total_bases']

print('La media de bicicletas libres en', origen, 'respecto a la hora elegida es:',int(stats.mean(bicisLibres)), '/', totalBasesPorEstacion[origen])
print('La media de bases libres en', destino ,'respecto a la hora estimada de llegada es:',int(stats.mean(basesLibres)), '/', totalBasesPorEstacion[destino])

La media de bicicletas libres en 57 respecto a la hora elegida es: 8 / 24
La media de bases libres en 38 respecto a la hora estimada de llegada es: 4 / 24
